In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import pandas_profiling
import logging
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json

import utils
 

/home/hussein/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
Using TensorFlow backend.


In [ ]:
import load_files_idPolair

In [ ]:
load_files_idPolair.data_station()

In [ ]:
d=pd.read_csv('data.csv')

In [ ]:
d.head(5)

In [ ]:
d.isnull().any().any()

In [ ]:
simple=pd.read_csv('simple.csv')

In [ ]:
simple

In [ ]:
simple.set_index(['index1','index2'],inplace=True)

In [ ]:
simple

In [ ]:
x=simple.groupby('index1').groups
k=x.keys()
k

In [ ]:
groups=simple.groupby('index1')
aa=groups.get_group(2)
aa.reset_index(level=0,inplace=True)
del aa['index1']
aa

In [ ]:
bb=groups.get_group(1)
bb.reset_index(level=0,inplace=True)
del bb['index1']
bb

In [ ]:
aa.join(bb, lsuffix='_aa', rsuffix='_bb')

In [ ]:
group=x[1]
group

In [6]:
def join_all_sites(path):
    data=pd.read_csv(path)
    data.set_index(['idPolair','date'],inplace=True)
    g=data.groupby('idPolair')
    del data
    groups=g.groups
    sites=groups.keys()
    result=None
    for site in sites:
        print(site)
        group=g.get_group(site)
        group.reset_index(level=0,inplace=True)
        del group['idPolair']
        if result is None:
            result=group
            result.columns = [str(col) + '_'+str(site) for col in result.columns]
        else:
            group.columns = [str(col) + '_'+str(site) for col in group.columns]
            result.join(group,how='outer',on='date',inplace=True)
        del group
    return result

In [ ]:
d=pd.read_csv('data.csv')
d.set_index(['idPolair','date'],inplace=True)


In [ ]:
new_data=f('data.csv')

In [1]:
import training_data


In [2]:
training_data.join_all_sites('data.csv')

7001.0
7004.0
7009.0
7010.0
7011.0
7012.0
7013.0


MemoryError: 

In [4]:
d=pd.read_csv('simple.csv')

In [7]:
d.set_index(['idPolair','date'],inplace=True)
d

x  y  z
idPolair date         
1        1     1  1  1
         2     1  2  1
         3     2  1  2
2        1     3  4  3
         2     3  4  3
         3     3  4  4
3        1     8  8  8
         2     9  9  9
         3     0  0  0

In [12]:
d['x'].shift(1)

idPolair  date
1         1       NaN
          2       1.0
          3       1.0
2         1       2.0
          2       3.0
          3       3.0
3         1       3.0
          2       8.0
          3       9.0
Name: x, dtype: float64

In [36]:
d=pd.read_csv('simple.csv')
del d['idPolair']
d.set_index('date',inplace=True)
d

,x,y,z
date,,,
1,1,1,1
2,1,2,1
3,2,1,2
1,3,4,3
2,3,4,3
3,3,4,4
1,8,8,8
2,9,9,9
3,0,0,0


In [37]:
d['x']

date
1    1
2    1
3    2
1    3
2    3
3    3
1    8
2    9
3    0
Name: x, dtype: int64

In [38]:
cols=d.columns
i=1
while(i<=3):
    for col in cols:
        d[col+'_t+'+str(i)]=d[col].shift(-i)
    i=i+1


In [87]:
def to_window(data,t):
    #data=pd.DataFrame(d)
    cols=data.columns
    i=1
    while(i<t):
        for col in cols:
            data[col+'_t+'+str(i)]=data[col].shift(-i)
        i=i+1
    data.dropna(inplace=True)
    return data

In [75]:
d=pd.read_csv('simple.csv')
d['idPolair']
d.set_index(['idPolair','date'],inplace=True)
#to_window(d,1)
d.

x  y  z
idPolair date         
1        1     1  1  1
         2     1  2  1
         3     2  1  2
2        1     3  4  3
         2     3  4  3
         3     3  4  4
3        1     8  8  8
         2     9  9  9
         3     0  0  0

In [74]:
to_window(d,1)

x  y  z  x_t+1  y_t+1  z_t+1  x_t+1_t+1  y_t+1_t+1  z_t+1_t+1
idPolair date                                                               
1        1     1  1  1    1.0    2.0    1.0        2.0        1.0        2.0
         2     1  2  1    2.0    1.0    2.0        3.0        4.0        3.0
         3     2  1  2    3.0    4.0    3.0        3.0        4.0        3.0
2        1     3  4  3    3.0    4.0    3.0        3.0        4.0        4.0
         2     3  4  3    3.0    4.0    4.0        8.0        8.0        8.0
         3     3  4  4    8.0    8.0    8.0        9.0        9.0        9.0

In [47]:
import training_data

In [54]:
d=training_data.join_all_sites('simple.csv')

1
2
3


In [53]:
d

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_1_t+1,...,z_3_t+1,x_1_t+2,y_1_t+2,z_1_t+2,x_2_t+2,y_2_t+2,z_2_t+2,x_3_t+2,y_3_t+2,z_3_t+2
date,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,3,4,3,8,8,8,1.0,...,9.0,2.0,1.0,2.0,3.0,4.0,4.0,0.0,0.0,0.0


In [60]:
to_window(d,2)

,x,y,z,x_t+1,y_t+1,z_t+1,x_t+2,y_t+2,z_t+2
date,,,,,,,,,
1,1,1,1,1.0,2.0,1.0,2.0,1.0,2.0
2,1,2,1,2.0,1.0,2.0,3.0,4.0,3.0
3,2,1,2,3.0,4.0,3.0,3.0,4.0,3.0
1,3,4,3,3.0,4.0,3.0,3.0,4.0,4.0
2,3,4,3,3.0,4.0,4.0,8.0,8.0,8.0
3,3,4,4,8.0,8.0,8.0,9.0,9.0,9.0
1,8,8,8,9.0,9.0,9.0,0.0,0.0,0.0


In [84]:
data=pd.read_csv('data.csv')
del data['idPolair']
data.set_index(['date'],inplace=True)

In [79]:
data.set_index(['date'],inplace=True)

In [ ]:
x=to_window(data,2)